# Neural Style Transfer

## Importing Libraries

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image as img
import pprint as pp

## Using a pre-trained VGG19 architecture

In [ ]:
img_size = 400
vgg19 = tf.keras.applications.VGG19(include_top=False,
                                        weights="Imagenet_weights\o",
                                        input_shape=(img_size, img_size, 3))
vgg19.trainable = False
print(vgg19.summary())

## Displaying the images

In [ ]:
content_image = img.open("Pictures\Content\Bridge.jpg")
print("Content's Image")
content_image

In [ ]:
style_image = img.open("Pictures\Style\Impressionism\Starry night.jpg")
print("Style's Image")
style_image

## Content Cost Function

In [ ]:
def content_cost(content_output, generated_output):
    c = content_output[-1]
    g = generated_output[-1]
    m, n_h, n_w, n_c = g.get_shape().as_list()
    c_unrolled = tf.reshape(c, shape=[m, n_h * n_w, n_c])
    g_unrolled = tf.reshape(g, shape=[m, n_h * n_w, n_c])
    j_content = (1 / (4 * n_h * n_w * n_c)) * tf.reduce_sum(tf.square(tf.subtract(c, g)))
    return j_content

## Gram Matrix

In [ ]:
def gram_matrix(x):
    gram = tf.matmul(x, tf.transpose(x))
    return gram

## Style Loss Function 

In [ ]:
def style_cost_layer(s, g):
    m, n_h, n_w, n_c = g.get_shape().as_list()
    s = tf.transpose(tf.reshape(s, shape=[-1, n_c]))
    g = tf.transpose(tf.reshape(g, shape=[-1, n_c]))
    gs = gram_matrix(s)
    gg = gram_matrix(g)
    j_style = tf.reduce_sum(tf.square(gs - gg)) / (4.0 *(( n_h * n_w * n_c) ** 2))
    return j_style

In [ ]:
for layer in vgg19.layers:
    print(layer.name)

## Retrieve a layer's output

In [ ]:
vgg19.get_layer('block5_conv4').output

## Define Style Layers

In [ ]:
style_layers = [
    ('block1_conv1', 0.2),
    ('block2_conv1', 0.2),
    ('block3_conv1', 0.2),
    ('block4_conv1', 0.2),
    ('block5_conv1', 0.2)]

## Compute style cost for several layers

In [ ]:
def style_cost(style_img_out, generated_img_out, sl = style_layers):
    j_style = 0
    s = style_img_out[:-1]
    g = generated_img_out[:-1]
    for i, weight in zip(range(len(s)), sl):
        j_style_layer = style_cost_layer(s[i], g[i])
        j_style += weight[1] * j_style_layer
    return j_style

## Total Cost

In [ ]:
@tf.function()
def total_cost(j_content, j_style, alpha = 10, beta = 40):
    J = alpha * j_content + beta * j_style
    return J

## Load the content image

In [ ]:
content_image = np.array(img.open("Pictures\Content\Bridge.jpg").resize((img_size, img_size)))
content_image = tf.constant(np.reshape(content_image, ((1,) + content_image.shape)))
print(content_image.shape)
plt.imshow(content_image[0])
plt.show()

## Load the style image

In [ ]:
style_image =  np.array(img.open("Pictures\Style\Impressionism\Starry night.jpg").resize((img_size, img_size)))
style_image = tf.constant(np.reshape(style_image, ((1,) + style_image.shape)))

print(style_image.shape)
plt.imshow(style_image[0])
plt.show()

## Random initialization of the generated image 

In [ ]:
generated_image = tf.Variable(tf.image.convert_image_dtype(content_image, tf.float32))
noise = tf.random.uniform(tf.shape(generated_image), -0.25, 0.25)
generated_image = tf.add(generated_image, noise)
generated_image = tf.clip_by_value(generated_image, clip_value_min=0.0, clip_value_max=1.0)

print(generated_image.shape)
plt.imshow(generated_image.numpy()[0])
plt.show()

## Load pre-trained VGG19 model

In [ ]:
def layer_outputs(vgg19, layer_names):
    outputs = [vgg19.get_layer(layer[0]).output for layer in layer_names]

    model = tf.keras.Model([vgg19.input], outputs)
    return model

In [ ]:
content_layer = [('block5_conv4', 1)]

vgg19_model_outputs = layer_outputs(vgg19, style_layers + content_layer)

In [ ]:
content_target = vgg19_model_outputs(content_image)
style_target = vgg19_model_outputs(style_image)

## Compute the content image

In [ ]:
preprocessed_content =  tf.Variable(tf.image.convert_image_dtype(content_image, tf.float32))
c = vgg19_model_outputs(preprocessed_content)

## Compute the Style image

In [ ]:
preprocessed_style =  tf.Variable(tf.image.convert_image_dtype(style_image, tf.float32))
s = vgg19_model_outputs(preprocessed_style)

In [ ]:
def clip_0_1(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

def tensor_to_image(tensor):
    tensor = tensor * 255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor) > 3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return img.fromarray(tensor)

## Compile step

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
@tf.function()
def train_step(generated_image):
    with tf.GradientTape() as tape:
        g = vgg19_model_outputs(generated_image)
        j_style = style_cost(s, g)
        j_content = content_cost(c, g)
        J = total_cost(j_content, j_style, 10, 40)
    grad = tape.gradient(J, generated_image)
    optimizer.apply_gradients([(grad, generated_image)])
    generated_image.assign(clip_0_1(generated_image))
    return J

In [ ]:
generated_image = tf.Variable(generated_image)

J1 = train_step(generated_image)
print(J1)

## Train step

In [ ]:
epochs = 10001
for i in range(epochs):
    train_step(generated_image)
    if i % 500 == 0:
        print(f"Epoch {i} ")
    if i % 500 == 0:
        image = tensor_to_image(generated_image)
        plt.imshow(image)
        image.save(f"Pictures\Output/image_{i}.jpg")
        plt.show() 

## Show images

In [ ]:
fig = plt.figure(figsize=(16, 4))
ax = fig.add_subplot(1, 3, 1)
plt.imshow(content_image[0])
ax.title.set_text('Content image')
ax = fig.add_subplot(1, 3, 2)
plt.imshow(style_image[0])
ax.title.set_text('Style image')
ax = fig.add_subplot(1, 3, 3)
plt.imshow(generated_image[0])
ax.title.set_text('Generated image')
plt.show()